# Apply Random Forest

In [1]:
library(grid)
library(rpart)
library(rpart.plot)
library(partykit)
library(lattice)
library(ggplot2)
library(caret)
library(e1071)
library(dplyr)
library(randomForest)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:dplyr’:

    combine

The following object is masked from ‘package:ggplot2’:

    margin



In [2]:
set.seed(1)

In [3]:
invalids <- c('Time', 'Longitude', 'Latitude', 'Brake', 'Accel', 'flag', 'Curve100', 'Curve150', 'TimeHeadway')
# invalids <- c('Time', 'Longitude', 'Latitude', 'Brake', 'Accel', 'flag', 'Curve100', 'Curve150')

In [4]:
valuables <- c("RoadType","CurveAverage","Speed", "Curve100", "Curve150","MaxSpeed","RiskFactor","Curve","DistSignal","Pitch","AheadDistance","AverageVelocity","TimeToCollision","AccelerationSpeed", "Engine", "SteeringAngle", "TimeHeadway", "Jerk", "LaneCount", "DiffAvgSpeed", "EmptinessOfRoad", "RoadFactor")

In [5]:
printf <- function(...) cat(sprintf(...))

In [21]:
trainAndPredict <- function(dfx, verbose=FALSE) {        
    train <- dfx[, c(valuables, "flag")]
    test <- dfx
    fit <- randomForest(flag ~ ., data=train, mtry=3, ntree=500)
    p <- predict(fit, newdata=test)

    showMatrix(test, p)

    nerr <- sum(p != test$flag)
}

In [34]:
showMatrix <- function(test, predictedFlags) {
    predictedRed <- test[predictedFlags == "Red", ]
    predictedBlue <- test[predictedFlags == "Blue", ]
    print(predictedRed %>% group_by(flag, AccelOrBrake) %>% summarize(count=n()))
    print(predictedBlue %>% group_by(flag, AccelOrBrake) %>% summarize(count=n()))
}

In [35]:
correctVsPredict <- function(test, predictedFlags) {    
    # for All
    # nTests <- nrow(test)
    # nCorrectAll <- sum((predictedFlags == test$flag) == TRUE)
    # printf("As fo All: correct/all = %d/%d = %f\n", nCorrectAll, nTests, nCorrectAll / nTests)
    
    # for Red
    predictedRedRows <- test[predictedFlags == "Red", ]
    nCorrectReds <- sum(predictedRedRows$flag == 'Red')
    nPredictedReds <- nrow(predictedRedRows)
    printf("As for Red: correct/predict = %d/%d = %f\n", nCorrectReds, nPredictedReds, nCorrectReds / nPredictedReds)
    
    # for Blue
    predictedBlueRows <- test[predictedFlags == "Blue", ]
    nCorrectBlues <- sum(predictedBlueRows$flag == 'Blue')
    nPredictedBlues <- nrow(predictedBlueRows)
    printf("As for Blue: correct/predict = %d/%d = %f\n", nCorrectBlues, nPredictedBlues, nCorrectBlues / nPredictedBlues)
    
    c(nCorrectReds/nPredictedReds, nCorrectBlues/nPredictedBlues)
}

In [36]:
printRedRatios <- function(dfx) {
    nRed <- nrow(dfx[dfx$flag == "Red", ])
    nAll <- nrow(dfx)
    printf("Red/All = %d/%d = %f\n", nRed, nAll, nRed/nAll)
    printf("1 - Red/All = %d/%d = %f\n", nAll - nRed, nAll, 1 - nRed/nAll)
}

In [37]:
createFormula <- function(invalids, allFeatures) {
    isValidColumn <- function(x) ! x %in% invalids
    
    features <- Filter(isValidColumn, allFeatures)
    return(paste("flag ~ ", paste(features, collapse=" + ")))
}

# Predict Red

In [38]:
df3 <- read.csv("../data/middle/sp5.csv", stringsAsFactors=FALSE)

In [40]:
df3$AccelOrBrake[df3$flag == "RedA"] <- "Accel"
df3$AccelOrBrake[df3$flag == "RedB"] <- "Brake"
df3$AccelOrBrake[df3$flag == "BlueA"] <- "Accel"
df3$AccelOrBrake[df3$flag == "BlueB"] <- "Brake"

In [41]:
df3$flag[df3$flag == "RedA"] <- "Red"
df3$flag[df3$flag == "RedB"] <- "Red"
df3$flag[df3$flag == "BlueA"] <- "Blue"
df3$flag[df3$flag == "BlueB"] <- "Blue"
df3$flag <- as.factor(df3$flag)

In [42]:
fomula <- paste("flag ~ ", paste(valuables, collapse=" + "))

In [43]:
set.seed(1)

In [44]:
df <- df3[, c(valuables, "AccelOrBrake", "flag")]

In [39]:
df3 %>% group_by(flag) %>% summarize(count=n())

flag,count
BlueA,77
BlueB,372
RedA,401
RedB,192


In [45]:
result <- trainAndPredict(df, verbose=TRUE)
# printRedRatios(df)
# printf("Red: Mean correct/predict = %f\n", result[2])
# printf("Blue: Mean correct/predict = %f\n", result[3])
# printf("CV value: %f", result[1])

Source: local data frame [2 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1    Red        Accel   401
2    Red        Brake   192
Source: local data frame [2 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue        Accel    77
2   Blue        Brake   372


In [19]:
set.seed(1)

In [20]:
# ntree.lim <- seq(50, 600, 30)

tune <- tune.randomForest(
        flag ~ .,
        data=dfx,
        ntree=ntree.lim,
        tunecontrol = tune.control(sampling = "cross", cross = 10)
)

ERROR: Error in tune.randomForest(flag ~ ., data = dfx, ntree = ntree.lim, tunecontrol = tune.control(sampling = "cross", :  オブジェクト 'ntree.lim' がありません 


In [ ]:
summary(tune)

In [ ]:
plot(tune)

In [ ]:
fit <- randomForest(flag ~ ., data=df, mtry=3, ntree=500)

In [ ]:
fit$importance